# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex consulting project').getOrCreate()
from pyspark.ml.regression import LinearRegression
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [2]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [3]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [4]:
data.select('Cruise_line').distinct().show()

+-----------------+
|      Cruise_line|
+-----------------+
|            Costa|
|              P&O|
|           Cunard|
|Regent_Seven_Seas|
|              MSC|
|         Carnival|
|          Crystal|
|           Orient|
|         Princess|
|        Silversea|
|         Seabourn|
| Holland_American|
|         Windstar|
|           Disney|
|        Norwegian|
|          Oceania|
|          Azamara|
|        Celebrity|
|             Star|
|  Royal_Caribbean|
+-----------------+



In [5]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index')
data = indexer.fit(data).transform(data)
data.select(['Cruise_line', 'Cruise_line_index']).distinct().show()

+-----------------+-----------------+
|      Cruise_line|Cruise_line_index|
+-----------------+-----------------+
|            Costa|              5.0|
|        Norwegian|              4.0|
|              MSC|              7.0|
|             Star|              8.0|
|           Orient|             19.0|
|           Disney|             17.0|
|Regent_Seven_Seas|             10.0|
|  Royal_Caribbean|              0.0|
|              P&O|              9.0|
|         Windstar|             14.0|
|         Princess|              2.0|
|         Seabourn|             13.0|
|          Azamara|             16.0|
| Holland_American|              3.0|
|          Oceania|             12.0|
|        Celebrity|              6.0|
|          Crystal|             18.0|
|        Silversea|             11.0|
|           Cunard|             15.0|
|         Carnival|              1.0|
+-----------------+-----------------+



In [7]:
used_features = data.select('Cruise_line_index', 
                            'Tonnage', 
                            'passengers', 
                            'length', 
                            'cabins', 
                            'passenger_density',
                            'crew')
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Cruise_line_index', 
                                       'Tonnage', 
                                       'passengers', 
                                       'length', 
                                       'cabins', 
                                       'passenger_density',
                                       'crew'], 
                            outputCol='features')
output = assembler.transform(used_features)
final_data = output.select('features', 'crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[16.0,30.27699999...|3.55|
|[16.0,30.27699999...|3.55|
|[1.0,47.262,14.86...| 6.7|
|[1.0,110.0,29.74,...|19.1|
|[1.0,101.353,26.4...|10.0|
|[1.0,70.367,20.52...| 9.2|
|[1.0,70.367,20.52...| 9.2|
|[1.0,70.367,20.56...| 9.2|
|[1.0,70.367,20.52...| 9.2|
|[1.0,110.23899999...|11.5|
|[1.0,110.0,29.74,...|11.6|
|[1.0,46.052,14.52...| 6.6|
|[1.0,70.367,20.52...| 9.2|
|[1.0,70.367,20.52...| 9.2|
|[1.0,86.0,21.24,9...| 9.3|
|[1.0,110.0,29.74,...|11.6|
|[1.0,88.5,21.24,9...|10.3|
|[1.0,70.367,20.52...| 9.2|
|[1.0,88.5,21.24,9...| 9.3|
|[1.0,70.367,20.52...| 9.2|
+--------------------+----+
only showing top 20 rows



In [8]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              113|
|   mean| 8.00548672566372|
| stddev|3.407177558151123|
|    min|              0.6|
|    max|             21.0|
+-------+-----------------+

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                45|
|   mean| 7.263555555555554|
| stddev|3.7208736483614584|
|    min|              0.59|
|    max|             12.38|
+-------+------------------+



In [9]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-5.32907051820075...|
|3.552713678800501...|
|6.217248937900877...|
|7.105427357601002...|
|7.105427357601002...|
|-2.66453525910037...|
|1.776356839400250...|
|-3.55271367880050...|
|-1.77635683940025...|
|3.552713678800501...|
|3.552713678800501...|
|3.552713678800501...|
|-2.66453525910037...|
|-1.77635683940025...|
|                 0.0|
|-5.32907051820075...|
|-3.55271367880050...|
|8.881784197001252...|
|-8.88178419700125...|
|7.993605777301127...|
+--------------------+
only showing top 20 rows



In [10]:
test_results.rootMeanSquaredError

3.752707105332666e-15

In [11]:
test_results.r2

1.0

In [12]:
test_results.meanSquaredError

1.4082810618414274e-29

In [13]:
test_results.meanAbsoluteError

3.0691498725193218e-15

In [14]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [15]:
from pyspark.sql.functions import corr
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|Cruise_line_index|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|5.063291139240507|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|

In [16]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [17]:
data.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

